# How To Train CNN Classifier With TF

This notebook demonstrates how to use `genomic_benchmarks` to train a neural network classifier on one of its benchmark datasets [human_nontata_promoters](https://github.com/ML-Bioinfo-CEITEC/genomic_benchmarks/tree/main/docs/human_nontata_promoters).

In [1]:
%load_ext autoreload
%autoreload 2

# Data download

With the function `download_dataset` downloads, we can download full-sequence form of the benchmark, splitted into train and test sets, one folder for each class.

In [3]:
from pathlib import Path
import tensorflow as tf

from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.models.tf import vectorize_layer
from genomic_benchmarks.models.tf import basic_cnn_model_v0 as model

if not is_downloaded('human_nontata_promoters'):
    download_dataset('human_nontata_promoters')

In [4]:
info('human_nontata_promoters', 0)

Dataset `human_nontata_promoters` has 2 classes: negative, positive.

All lenghts of genomic intervals equals 251.

Totally 36131 sequences have been found, 27097 for training and 9034 for testing.


,train,test
negative,12355,4119
positive,14742,4915


## TF Dataset object

To train the model with TensorFlow, we must create a TF Dataset. Because the directory structure of our benchmarks is ready for training, we can just call `tf.keras.preprocessing.text_dataset_from_directory` function as follows.

In [5]:
BATCH_SIZE = 64
SEQ_PATH = Path.home() / '.genomic_benchmarks' / 'human_nontata_promoters'
CLASSES = [x.stem for x in (SEQ_PATH/'train').iterdir() if x.is_dir()]

train_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'train',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

Found 27097 files belonging to 2 classes.


## Text vectorization

To convert the strings to tensors, we internally use TF `TextVectorization` layer and splitting to characters.

In [6]:
vectorize_layer.adapt(train_dset.map(lambda x, y: x))
#vectorize_layer.set_vocabulary(vocab=np.asarray(['a', 'c', 't', 'g', 'n']))
vectorize_layer.get_vocabulary()

2021-10-22 16:02:38.901442: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


['', '[UNK]', 'g', 'c', 't', 'a']

In [7]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = train_dset.map(vectorize_text)

## Model training

To get a baseline (other models can be compared to) we ship a package with [a simple CNN model](https://github.com/ML-Bioinfo-CEITEC/genomic_benchmarks/blob/main/src/genomic_benchmarks/models/tf.py). We have vectorized the dataset before training the model to speed up the process.

In [8]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [9]:
EPOCHS = 10

history = model.fit(
    train_ds,
    epochs=EPOCHS)

Epoch 1/10
424/424 [==============================] - 7s 16ms/step - loss: 0.4556 - binary_accuracy: 0.7943
Epoch 2/10
424/424 [==============================] - 7s 17ms/step - loss: 0.3847 - binary_accuracy: 0.8292
Epoch 3/10
424/424 [==============================] - 7s 17ms/step - loss: 0.3633 - binary_accuracy: 0.8388
Epoch 4/10
424/424 [==============================] - 7s 17ms/step - loss: 0.3444 - binary_accuracy: 0.8463
Epoch 5/10
424/424 [==============================] - 8s 19ms/step - loss: 0.3264 - binary_accuracy: 0.8572
Epoch 6/10
424/424 [==============================] - 8s 20ms/step - loss: 0.3108 - binary_accuracy: 0.8642
Epoch 7/10
424/424 [==============================] - 8s 20ms/step - loss: 0.2954 - binary_accuracy: 0.8721
Epoch 8/10
424/424 [==============================] - 8s 19ms/step - loss: 0.2851 - binary_accuracy: 0.8784
Epoch 9/10
424/424 [==============================] - 8s 19ms/step - loss: 0.2741 - binary_accuracy: 0.8843
Epoch 10/10
424/424 [=======

## Evaluation on the test set

Finally, we can do the same pre-processing for the test set and evaluate the accuracy of our model.

In [12]:
test_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'test',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

test_ds =  test_dset.map(vectorize_text)

Found 9034 files belonging to 2 classes.


In [13]:
model.evaluate(test_ds)

142/142 [==============================] - 1s 5ms/step - loss: 0.3401 - binary_accuracy: 0.8493


[0.34010663628578186, 0.849346935749054]